
# EEG Artifact removal using ATAR

ATAR: Automatic and Tunable Artifact Removal Algorithm

**ATAR Algorithm -  Automatic and Tunable Artifact Removal Algorithm for EEG Signal.** 


    Apply ATAR on short windows of signal (single channel):

    Signal is decomposed in smaller overlapping windows and reconstructed after correcting using overlap-add method.

    For more details, check [1]_

    **Operating Modes**

    **1. Soft Thresholding**

    .. math ::

       \lambda_s (w)  &=  w  & \quad \text{if } |w|<\theta_{\gamma}

          &=  \theta_{\alpha} \frac{1 - e^{\alpha w}}{1 + e^{\alpha w}}   & \quad \text{otherwise}


    **2. Elimination**

    .. math ::

       \lambda_e (w)  &=  w  & \quad \text{if } |w| \le \theta_{\alpha}

         &=  0   & \quad \text{otherwise}


    **3. Linear Attenuation**
    
    .. math ::

       \lambda_a (w)  &=  w  & \quad \text{if } |w| \le \theta_{\alpha}

              &=  sgn(w) \theta_{\alpha} \Big( 1 -  \frac{|w| - \theta_{\alpha}}{\theta_{\beta}-\theta_{\alpha}} \Big) & \quad \text{if } \theta_{\alpha} < |w| \le \theta_{\beta}
              
              &=  0   & \quad \text{otherwise}


    **Computing Threshold**

    * $f_{\beta}(r)  = k_2 \cdot exp \Big(-\beta \frac{w_{max}}{k_2} \times \frac{r}{2} \Big)$

    * $\theta_{\alpha}  =  f_{\beta}(r)  \quad \text{if } f_{\beta}(r) \ge k_1$ otherwise $\theta_{\alpha}  =  k_1$

    * $\theta_{\gamma}  = g_f \times \theta_{\alpha}$ ,  where a default value for 'g_f = 0.8' **For Soft-threshold**
    
    * $\theta_{\beta}  = b_f \times \theta_{\alpha}$ , where a default value for 'b_f = 2' **For Linear Attenuation**


In [ ]:
# Importing libraries/spkit
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import spkit as sp
print('spkit version :', sp.__version__)

## Load and filter EEG Signal



In [ ]:
# Load sample EEG Data ( 16 sec, 128 smapling rate, 14 channel)
# Filter signal (with IIR highpass 0.5Hz)

X, fs, ch_names = sp.data.eeg_sample_14ch()
Xf = sp.filter_X(X.copy(),fs=128.0, band=[0.5], btype='highpass',ftype='SOS')
print(Xf.shape)

Artifact removal using ATAR: Single Channel




In [ ]:
x = Xf[:,0] 
xc1 = sp.eeg.ATAR(x,wv='db3', winsize=128, thr_method='ipr',beta=0.1, k1=10, k2=100,OptMode='soft',verbose=1)
xc2 = sp.eeg.ATAR(x,wv='db3', winsize=128, thr_method='ipr',beta=0.1, k1=10, k2=100,OptMode='linAtten',verbose=1)
xc3 = sp.eeg.ATAR(x,wv='db3', winsize=128, thr_method='ipr',beta=0.1, k1=10, k2=100,OptMode='elim',verbose=1)

Artifact removal using ATAR: Multi-Channel




In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0)
print(XR.shape)

## Plots



In [ ]:
t = np.arange(Xf.shape[0])/fs

plt.figure(figsize=(10,5))
plt.subplot(111)
plt.plot(t,x, label='$x$: raw EEG - single channel')
plt.plot(t,xc1,label=r'$x_{c1}$: Soft Thresholding')
plt.plot(t,xc2,label=r'$x_{c2}$: Linear Attenuation')
plt.plot(t,xc3,label=r'$x_{c3}$: Elimination')
plt.xlim([9,12])
plt.ylim([-200,200])
plt.legend(bbox_to_anchor=(0.5,0.99),ncol=2,fontsize=8)
plt.grid()
plt.title(r'ATAR Algorithm')
plt.xlabel('time (s)')
plt.show()

plt.figure(figsize=(12,8))
plt.subplot(211)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(212)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Soft Thresholding (default mode)')
plt.tight_layout()
plt.grid()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(t,Xf+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf: 14 channel - EEG Signal (filtered)')
plt.tight_layout()
plt.grid()
plt.show()

## ATAR: Linear Attenuation Mode



In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0,OptMode='linAtten')
print(XR.shape)

t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(122)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Linear Attenuation Mode')
plt.tight_layout()
plt.show()

## ATAR - Elimination Mode



In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0,OptMode='elim')
print(XR.shape)

t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(122)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Elimination Mode')
plt.tight_layout()
plt.show()